In [ ]:
#--------------------------------------------------------------------------------------------------------
# filename: 7_check_if_pv_in_image.ipynb
# description: find images with photovoltaik installations
# Author: ChatGPT, 2023
# last updated: 08.03.23
#--------------------------------------------------------------------------------------------------------
#------------------------------------------------------#
#  THIS SECTION WAS CREATED BY CHATGPT (OpenAi, 2023) #
#----------------------------------------------------#
import os
import pandas as pd
from osgeo import gdal
from rtree import index

class PV_Finder:
    """
    Class to find images with Photovoltaik panels.
    Attributes:
        folder_path (str): Path to the folder containing the images in tiff format.
        with_pv_folder (str): Path to the folder where the images with Photovoltaik panels will be saved.
        coordinates_file (str): Path to the csv file containing the coordinates of Photovoltaik panels.
    """
    def __init__(self, folder_path, with_pv_folder, coordinates_file):
        self.folder_path = folder_path
        self.with_pv_folder = with_pv_folder
        self.coordinates_file = coordinates_file
        
        
    def create_csv_file(self, pv_list):
        """
        create_csv_file method:
        This method converts a list of filenames with PV present to a DataFrame and saves it to a CSV file.
        Inputs:
            pv_list: list of filenames with PV present
        Outputs:
            A CSV file saved in the specified file path.
        """
        df = pd.DataFrame(pv_list)
        # Save the DataFrame to a CSV file
        df.to_csv('../../_data/geodata/pv_list.csv', index=False, header=False)

        
    def find_images(self):
        """
        Find images with Photovoltaik panels and save them to a separate folder.
        """
        # Read in the pandas data set
        df = pd.read_csv(self.coordinates_file)

        new_df = pd.DataFrame()
        pv_found = []
        # Create the spatial index
        spatial_index = index.Index()
        for idx, row in df.iterrows():
            spatial_index.insert(idx, (row['_x'], row['_y'], row['_x'], row['_y']))

        for subdir, dirs, files in os.walk(self.folder_path):
            for file in files:
                if file.endswith(".tif"): 
                    image_path = os.path.join(subdir, file)
                    subdir_name = os.path.basename(subdir)
                    ds = gdal.Open(image_path)

                    # Get the coordinates of the image
                    x_min, x_res, x_skew, y_max, y_skew, y_res = ds.GetGeoTransform()
                    x_max = x_min + ds.RasterXSize * x_res
                    y_min = y_max + ds.RasterYSize * y_res

                    # Use the spatial index to check if any coordinates in the data set intersect with the image
                    intersecting_coordinates = list(spatial_index.intersection((x_min, y_min, x_max, y_max)))

                    # Do something with the intersecting coordinates, for example, save them to a new data frame
                    if intersecting_coordinates:
                        value = df.loc[intersecting_coordinates]
                        new_df=new_df.append([value])     

                        # create folder "with_pv" if not exist                         
                        subfolder_path = os.path.join(self.with_pv_folder, subdir_name)
                        if not os.path.exists(subfolder_path):
                            os.makedirs(subfolder_path)

                        # make copy of image and insert them into the subfolder "with_pv"                       
                        os.system(f"cp {image_path} {subfolder_path}")
                        print("PV found in" , file, "!" )

                        # append index of coordinates found to list pv_found
                        pv_found.append(file)

                        # create csv with coordinates for each tif folder   
                        new_df.to_csv(f"../../_data/geodata/pv_tif/{subdir_name}_intersecting_coordinates.csv")

        # create csv from list pv_found     
        self.create_csv_file(pv_found)
        print("All images with PV-installation have been sorted out.")
    

# Initialize the ImageProcessor object and call the process_images method
finder = PV_Finder("/Volumes/HDD_6TB/tiled_images/rgb_images", 
                   "/Volumes/HDD_6TB/tiled_images/with_pv", 
                   "/Volumes/HDD_6TB/geodata/df_pv_cleaned.csv")


finder.find_images()